In [ ]:
import pathlib

import pandas as pd
from IPython.display import display
import yaml

import dlsproc.xml
import dlsproc.bundle
import dlsproc.hier
import dlsproc.io
import dlsproc.parse
import dlsproc.postprocess

So that *all* the columns of a `pd.DataFrame` are shown:

In [ ]:
pd.set_option('display.max_columns', None)

The directory containing downloaded data:

In [ ]:
data_directory = pathlib.Path.cwd().parent / 'data'
assert data_directory.exists()
print(data_directory)

/home/manu/dlsproc/data


## Outsiders

> Licitaciones publicadas en la Plataforma mediante mecanismos de agregación, excluyendo los contratos menores

A file to *cache* the read data

In [ ]:
outsiders_output_file = dlsproc.io.File('outsiders')

The directory for *outsiders* data

In [ ]:
outsiders_directory = data_directory / 'agregados'
assert outsiders_directory.exists()
print(outsiders_directory)

/home/manu/dlsproc/data/agregados


The file belown was downloaded [here](https://www.hacienda.gob.es/es-ES/GobiernoAbierto/Datos%20Abiertos/Paginas/LicitacionesAgregacion.aspx).

In [ ]:
outsiders_zip_file = outsiders_directory / 'PlataformasAgregadasSinMenores_2021.zip'
assert outsiders_zip_file.exists()

The data is read from a file if present, or from the given **zip** if not

In [ ]:
if outsiders_output_file.exists():
    outsiders_df = outsiders_output_file.read()
else:
    # data is read from the given zip, and concatenated into a single `pd.DataFrame`...
    outsiders_df = dlsproc.bundle.read_zip(outsiders_zip_file, concatenate=True)
    
    # ...re-structured with multiindexed columns...
    outsiders_df = dlsproc.hier.flat_df_to_multiindexed_df(outsiders_df)
    
    # ...and saved
    outsiders_output_file.write(outsiders_df)

How many entries and how many fields?

In [ ]:
outsiders_df.shape

(208204, 40)

Examples of columns

In [ ]:
outsiders_df.columns.values[10:20]

array([('ContractFolderStatus', 'ProcurementProject', 'BudgetAmount', 'EstimatedOverallContractAmount', '', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'BudgetAmount', 'TaxExclusiveAmount', '', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'RequiredCommodityClassification', 'ItemClassificationCode', '', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'RealizedLocation', 'CountrySubentityCode', '', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'PlannedPeriod', 'DurationMeasure', '', '', ''),
       ('ContractFolderStatus', 'TenderingProcess', 'ProcedureCode', '', '', '', ''),
       ('ContractFolderStatus', 'TenderingProcess', 'TenderSubmissionDeadlinePeriod', 'EndDate', '', '', ''),
       ('ContractFolderStatus', 'TenderingProcess', 'TenderSubmissionDeadlinePeriod', 'EndTime', '', '', ''),
       ('ContractFolderStatus', 'ValidNoticeInfo', 'NoticeTypeCode', '', '', '', ''),
       ('ContractFolderStatus', 'ValidNotice

The first few rows

In [ ]:
outsiders_df.head()

id  \
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
file name                           entry                                                      
PlataformasAgregadasSinMenores.atom 0      https://contrataciondelestado.es/sindicacion/P...   
                                    1      https://contrataciondelestado.es/sindicacion/P...   
                                    2      https://contrataciondelestado.es/sindicacion/P...   
                                    3      https://contrataciondelestado.es/sindicacion/P...   
                                    4      https://contrataciondelestado.es/sindicacion/P...   

                                                                                     summary  \
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
file name                           entry                                                      
PlataformasAgregadasSinMenores.atom 0      Id licitación: 3064_23/2021; Órgano de Contrat...   
                                    1      Id licitación: 3051_CONT_SERVICIOS/2021/40; Ór...   
                                    2      Id licitación: 3091_21/2021; Órgano de Contrat...   
                                    3      Id licitación: 3069_1/2021; Órgano de Contrata...   
                                    4      Id licitación: 8044_30146-0-21; Órgano de Cont...   

                                                                                       title  \
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
file name                           entry                                                      
PlataformasAgregadasSinMenores.atom 0      Contrato del Servicio para el desarrollo de la...   
                                    1      REDACCIÓN DEL PLAN ESTRATÉGICO DE ACCESIBILIDA...   
                                    2      Contratación de obra de línea subterránea en A...   
                                    3      Construcción de depósito regulador de 100m3 en...   
                                    4          Renovación de la cubierta de edificio docente   

                                                                   updated  \
                                                                             
                                                                             
                                                                             
                               

Some types

In [ ]:
outsiders_df.dtypes[:20]

id                                                                                                                                        string
summary                                                                                                                                   string
title                                                                                                                                     string
updated                                                                                                                      datetime64[ns, UTC]
ContractFolderStatus  ContractFolderID                                                                                                    string
                      ContractFolderStatusCode                                                                                            string
                      LocatedContractingParty   Party                            PartyName                       Name             

There are *multivalued* columns.

In [ ]:
dlsproc.structure.multivalued_columns(outsiders_df)

[('ContractFolderStatus',
  'ProcurementProject',
  'RequiredCommodityClassification',
  'ItemClassificationCode',
  '',
  '',
  ''),
 ('ContractFolderStatus', 'ValidNoticeInfo', 'NoticeTypeCode', '', '', '', ''),
 ('ContractFolderStatus',
  'ValidNoticeInfo',
  'AdditionalPublicationStatus',
  'PublicationMediaName',
  '',
  '',
  ''),
 ('ContractFolderStatus',
  'ValidNoticeInfo',
  'AdditionalPublicationStatus',
  'AdditionalPublicationDocumentReference',
  'IssueDate',
  '',
  ''),
 ('ContractFolderStatus', 'TenderResult', 'ResultCode', '', '', '', ''),
 ('ContractFolderStatus',
  'TenderResult',
  'ReceivedTenderQuantity',
  '',
  '',
  '',
  ''),
 ('ContractFolderStatus',
  'TenderResult',
  'WinningParty',
  'PartyIdentification',
  'ID',
  '',
  ''),
 ('ContractFolderStatus',
  'TenderResult',
  'WinningParty',
  'PartyName',
  'Name',
  '',
  ''),
 ('ContractFolderStatus',
  'TenderResult',
  'AwardedTenderedProject',
  'LegalMonetaryTotal',
  'TaxExclusiveAmount',
  '',
  '')

### Entity

*Multiindex* column names are built for

- *Entity*

In [ ]:
entity_col = dlsproc.hier.pad_col_levels(outsiders_df, ['ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PartyName', 'Name'])
entity_col

('ContractFolderStatus',
 'LocatedContractingParty',
 'Party',
 'PartyName',
 'Name',
 '',
 '')

- *Amount*

In [ ]:
amount_col = dlsproc.hier.pad_col_levels(outsiders_df, ['ContractFolderStatus', 'ProcurementProject', 'BudgetAmount', 'TaxExclusiveAmount'])
amount_col

('ContractFolderStatus',
 'ProcurementProject',
 'BudgetAmount',
 'TaxExclusiveAmount',
 '',
 '',
 '')

- *Winner*

In [ ]:
winner_col = dlsproc.hier.pad_col_levels(outsiders_df, ['ContractFolderStatus', 'TenderResult', 'WinningParty', 'PartyName', 'Name'])
winner_col

('ContractFolderStatus',
 'TenderResult',
 'WinningParty',
 'PartyName',
 'Name',
 '',
 '')

Number of projects per entity along with the *accumulated* amount of money

In [ ]:
number_amount_df = outsiders_df[[entity_col, amount_col]].groupby(entity_col).agg(amount=(amount_col, 'sum'), number=(amount_col, 'count'))
number_amount_df

,amount,number
"(ContractFolderStatus, LocatedContractingParty, Party, PartyName, Name, , )",,
AGENCIA NAVARRA DE AUTONOMÍA Y DESARROLLO DE L...,1.040800e+07,8
AGENCIA NAVARRA PARA LA AUTONOMIA Y DESARROLLO...,1.796660e+06,2
AJUNTAMENT DE CERVIÀ DE TER,2.198030e+05,4
AJUNTAMENT DE FALSET,2.123535e+06,9
AJUNTAMENT DE SANT JAUME DELS DOMENYS,2.495736e+07,25
...,...,...
presidencia,3.065700e+05,6
Àrea Metropolitana de Barcelona,9.364749e+09,1410
ÁREA DE SALUD I (MURCIA OESTE - H.U. VIRGEN DE...,4.743750e+04,1


Entities with the biggest amounts

In [ ]:
number_amount_df.sort_values('amount', ascending=False)

,amount,number
"(ContractFolderStatus, LocatedContractingParty, Party, PartyName, Name, , )",,
Servicio Andaluz de Salud,2.757164e+10,3316
Consejería de Vivienda y Administración Local,2.476717e+10,38
"Canal de Isabel II, S.A.",1.622211e+10,1850
"Empresa Pública de Metro de Madrid, S.A.",1.288923e+10,3323
Ajuntament de Barcelona,1.030976e+10,2380
...,...,...
"ZUGAZTEL, M. P., S. A.",0.000000e+00,0
Directora Gerente Agencia Navarra de Autonomía...,0.000000e+00,0
"ITELAZPI, S.A.",0.000000e+00,0


### Documents

Columns containing *DocumentReference* in the name, allegedly URLs to documents. For a clearer visualization, some levels of the rows' `pd.MultiIndex` are dropped.

In [ ]:
# if zip (source compressed file) is a "level" of the index...
if 'zip' in outsiders_df.index.names:
    display(outsiders_df.loc[:, dlsproc.hier.columns_containing(outsiders_df, 'DocumentReference')].droplevel(['zip', 'file name']))
# if zip is NOT a "level" of the index...
else:
    display(outsiders_df.loc[:, dlsproc.hier.columns_containing(outsiders_df, 'DocumentReference')].droplevel(['file name']))

ContractFolderStatus  \
                                         ValidNoticeInfo   
                             AdditionalPublicationStatus   
                  AdditionalPublicationDocumentReference   
                                               IssueDate   
                                                           
                                                           
entry                                                      
0                                             2021-12-31   
1                                             2022-01-01   
2                               [2021-12-17, 2021-12-31]   
3      [2021-12-16, 2021-12-16, 2021-12-16, 2021-12-1...   
4       [2021-11-24, 2021-12-10, 2021-12-16, 2021-12-31]   
..                                                   ...   
463                                           2021-11-26   
464                                           2021-11-25   
465                             [2021-11-29, 2021-11-30]   
466                 [2021-11-19, 2021-11-22, 2021-12-29]   
467                             [2021-12-07, 2021-12-21]   

                                                          \
                                  LegalDocumentReference   
                                                      ID   
                                                           
                                                           
                                                           
                                                           
entry                                                      
0                                                   <NA>   
1                                                   <NA>   
2                                                   <NA>   
3                                                   <NA>   
4                                                   <NA>   
..                                                   ...   
463    2021.63 PCAP Reforma interior sede IRUÑA EITB.pdf   
464                2021.67 PCAP Suministro de Prensa.pdf   
465                                            PCAP.docx   
466                    3- PCP - LicenciasCV_2022_v11.pdf   
467                                             PCAP.doc   

                                                          \
                                                           
                                              Attachment   
                                       ExternalReference   
                                                     URI   
                                                           
                                                           
entry                                                      
0                                                   <NA>   
1                                                   <NA>   
2                                                   <NA>   
3                                                   <NA>   
4                                                   <NA>   
..                                                   ...   
463    https://www.contratacion.euskadi.eus/ac70cPubl...   
464    https://www.contratacion.euskadi.eus/ac70cPubl...   
465    https://www.contratacion.euskadi.eus/ac70cPubl...   
466    https://www.contratacion.euskadi.eus/ac70cPubl...   
467    https://www.contratacion.euskadi.eus/ac70cPubl...   

                                                          \
                              TechnicalDocumentReference   
                                                      ID   
                                                           
                                                           
                                                           
                                                           
entry                                                      
0                                                   <NA>   
1                                                   <NA>   
2            

### Duplicates

In [ ]:
only_last_update_df = dlsproc.postprocess.keep_updates_only(outsiders_df)
only_last_update_df.head(2)

id  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20210101_030039.... 294    https://contrataciondelestado.es/sindicacion/P...   
                                                   291    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                    summary  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20210101_030039.... 294    Id licitación: PA-02-2020;  Órgano de Contrata...   
                                                   291    Id licitación: 2006OB07; Órgano de Contratació...   

                                                                                                      title  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20210101_030039.... 294    SERVICIO DE RECEPCIÓN Y ATENCIÓN TELEFÓNICA A ...   
                                                   291    Subministrament d'energia elèctrica en BAIXA T...   

                                                                                  updated  \
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
file name                                          entry                                    
PlataformasAgregadasSinMenor

Size of the above `pd.DataFrame`

In [ ]:
only_last_update_df.shape

(64020, 40)

For the original `pd.DataFrame`

In [ ]:
outsiders_df.shape

(208204, 40)

A `pd.Series` yielding entries that were deleted and when (indexed by *file name* and `entry`)

In [ ]:
outsiders_deleted_series = dlsproc.bundle.read_deleted_zip(outsiders_zip_file)
outsiders_deleted_series

file name                                              id                                                                                 
PlataformasAgregadasSinMenores.atom                    https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/8231713    2022-01-01 00:12:28.323000+01:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/8063576    2022-01-01 00:12:28.255000+01:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/7824334    2022-01-01 00:12:28.189000+01:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/8163656    2022-01-01 00:12:28.143000+01:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/6834832    2022-01-01 00:12:28

Duplicates are dropped. In order to do so it is convenient to turn the `pd.Series` into a `pd.DataFrame` by calling `reset_index`, which turns the *multiindex* into columns

In [ ]:
# deduplicated_outsiders_deleted_df = outsiders_deleted_series.reset_index().drop_duplicates('entry')
deduplicated_outsiders_deleted_df = outsiders_deleted_series.reset_index().drop_duplicates('id').set_index(['file name', 'id'])
deduplicated_outsiders_deleted_df

deleted_on
file name                                          id                                                                                  
PlataformasAgregadasSinMenores.atom                https://contrataciondelestado.es/sindicacion/Pl...  2022-01-01 00:12:28.323000+01:00
                                                   https://contrataciondelestado.es/sindicacion/Pl...  2022-01-01 00:12:28.255000+01:00
                                                   https://contrataciondelestado.es/sindicacion/Pl...  2022-01-01 00:12:28.189000+01:00
                                                   https://contrataciondelestado.es/sindicacion/Pl...  2022-01-01 00:12:28.143000+01:00
                                                   https://contrataciondelestado.es/sindicacion/Pl...  2022-01-01 00:12:28.105000+01:00
...                                                                                                                                 ...
PlataformasAgregadasSinMenores_20211231_030012_... https://contrataciondelestado.es/sindicacion/Pl...  2021-12-30 00:12:32.408000+01:00
                                                   https://contrataciondelestado.es/sindicacion/Pl...  2021-12-30 00:12:32.358000+01:00
                                                   https://contrataciondelestado.es/sindicacion/Pl...  2021-12-30 00:12:32.316000+01:00
                                                   https://contrataciondelestado.es/sindicacion/Pl...  2021-12-30 00:12:32.267000+01:00
                                                   https://contrataciondelestado.es/sindicacion/Pl...  2021-12-29 14:12:30.201000+01:00

[8092 rows x 1 columns]

Number of duplicates

In [ ]:
n_duplicate_deletes = len(outsiders_deleted_series) - len(deduplicated_outsiders_deleted_df)
n_duplicate_deletes

141

The `pd.DataFrame` in which only the last update is kept is (left-)joined with the series yielding deleted entries. The result is a *stateful* `pd.DataFrame` in the sense that, for every entry, we know its state: deleted or not. Notice that `only_last_update_df`'s index is reset for easying the merge (because every contract only shows up once in `only_last_update_df`, *file name* and *id* should still provide a unique index...though it probably doesn't matter anyway.)


**Columns `pd.MultiIndex`  is lost**.

In [ ]:
stateful_outsiders_df = only_last_update_df.reset_index().set_index(['file name', 'id']).merge(deduplicated_outsiders_deleted_df, how='left', on=['file name', 'id'])
# stateful_outsiders_df = only_last_update_df.merge(deduplicated_outsiders_deleted_df, how='left', left_on='id', right_on='entry')
stateful_outsiders_df.head()

/tmp/ipykernel_3167471/3197343437.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (7 levels on the left, 1 on the right)
  stateful_outsiders_df = only_last_update_df.reset_index().set_index(['file name', 'id']).merge(deduplicated_outsiders_deleted_df, how='left', on=['file name', 'id'])


(entry, , , , , , )  \
file name                                          id                                                                        
PlataformasAgregadasSinMenores_20210101_030039.... https://contrataciondelestado.es/sindicacion/Pl...                  294   
                                                   https://contrataciondelestado.es/sindicacion/Pl...                  291   
                                                   https://contrataciondelestado.es/sindicacion/Pl...                  288   
                                                   https://contrataciondelestado.es/sindicacion/Pl...                  282   
                                                   https://contrataciondelestado.es/sindicacion/Pl...                  281   

                                                                                                                                   (summary, , , , , , )  \
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20210101_030039.... https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: PA-02-2020;  Órgano de Contrata...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 2006OB07; Órgano de Contratació...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: PRO/2020/168; Órgano de Contrat...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 5/2020; Órgano de Contratación:...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: PRO/2020/169; Órgano de Contrat...   

                                                                                                                                     (title, , , , , , )  \
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20210101_030039.... https://contrataciondelestado.es/sindicacion/Pl...  SERVICIO DE RECEPCIÓN Y ATENCIÓN TELEFÓNICA A ...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Subministrament d'energia elèctrica en BAIXA T...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  4a Pròrroga del contracte de la licitació PSP8...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Servei de recollida porta a porta i transport ...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Pròrroga del contracte de la licitació PSP81/1...   

                                                                                                                 (updated, , , , , , )  \
file name                                          id                                                                                    
PlataformasAgregadasSinMenores_20210101_030039.... https://contrataciondelestado.es/sindicacion/Pl... 2020-12-30 14:25:27.661000+00:00   
                                                   https://contrataciondelestado.es/sindicacion/Pl... 2020-12-30 14:26:25.241000+00:00   
                                                   https://contrataciondelestado.es/sindicacion/Pl... 2020-12-30 14:41:38.851000+00:00   
                                                   https://contrataciondelestado.es/sindicacion/Pl... 2020-12-30 14:41:39.165000+00:00   
                                                   https://contrataciondelestado.es/sindicacion/Pl... 2020-12-30 14:41:39.223000+00:00   

                                                                   

The new *merged* `pd.DataFrame` has the same number of rows as the original one

In [ ]:
assert len(stateful_outsiders_df) == len(only_last_update_df)

How many entries have been deleted?

In [ ]:
stateful_outsiders_df['deleted_on'].notna().sum()

15

Some examples

In [ ]:
stateful_outsiders_df[stateful_outsiders_df['deleted_on'].notna()].head()

,,"(entry, , , , , , )","(summary, , , , , , )","(title, , , , , , )","(updated, , , , , , )","(ContractFolderStatus, ContractFolderID, , , , , )","(ContractFolderStatus, ContractFolderStatusCode, , , , , )","(ContractFolderStatus, LocatedContractingParty, Party, PartyName, Name, , )","(ContractFolderStatus, LocatedContractingParty, ParentLocatedParty, PartyName, Name, , )","(ContractFolderStatus, ProcurementProject, Name, , , , )","(ContractFolderStatus, ProcurementProject, TypeCode, , , , )","(ContractFolderStatus, ProcurementProject, BudgetAmount, EstimatedOverallContractAmount, , , )","(ContractFolderStatus, ProcurementProject, BudgetAmount, TaxExclusiveAmount, , , )","(ContractFolderStatus, ProcurementProject, RequiredCommodityClassification, ItemClassificationCode, , , )","(ContractFolderStatus, ProcurementProject, RealizedLocation, CountrySubentityCode, , , )","(ContractFolderStatus, ProcurementProject, PlannedPeriod, DurationMeasure, , , )","(ContractFolderStatus, TenderingProcess, ProcedureCode, , , , )","(ContractFolderStatus, TenderingProcess, TenderSubmissionDeadlinePeriod, EndDate, , , )","(ContractFolderStatus, TenderingProcess, TenderSubmissionDeadlinePeriod, EndTime, , , )","(ContractFolderStatus, ValidNoticeInfo, NoticeTypeCode, , , , )","(ContractFolderStatus, ValidNoticeInfo, AdditionalPublicationStatus, PublicationMediaName, , , )","(ContractFolderStatus, ValidNoticeInfo, AdditionalPublicationStatus, AdditionalPublicationDocumentReference, IssueDate, , )","(ContractFolderStatus, ProcurementProject, PlannedPeriod, StartDate, , , )","(ContractFolderStatus, ProcurementProject, PlannedPeriod, EndDate, , , )","(ContractFolderStatus, LocatedContractingParty, ParentLocatedParty, ParentLocatedParty, PartyName, Name, )","(ContractFolderStatus, LegalDocumentReference, ID, , , , )","(ContractFolderStatus, LegalDocumentReference, Attachment, ExternalReference, URI, , )","(ContractFolderStatus, TechnicalDocumentReference, ID, , , , )","(ContractFolderStatus, TechnicalDocumentReference, Attachment, ExternalReference, URI, , )","(ContractFolderStatus, TenderResult, ResultCode, , , , )","(ContractFolderStatus, TenderResult, ReceivedTenderQuantity, , , , )","(ContractFolderStatus, TenderResult, WinningParty, PartyIdentification, ID, , )","(ContractFolderStatus, TenderResult, WinningParty, PartyName, Name, , )","(ContractFolderStatus, TenderResult, AwardedTenderedProject, LegalMonetaryTotal, TaxExclusiveAmount, , )","(ContractFolderStatus, TenderingProcess, ParticipationRequestReceptionPeriod, EndDate, , , )","(ContractFolderStatus, TenderingProcess, ParticipationRequestReceptionPeriod, EndTime, , , )","(ContractFolderStatus, LocatedContractingParty, BuyerProfileURIID, , , , )","(ContractFolderStatus, LocatedContractingParty, Party, PartyIdentification, ID, , )","(ContractFolderStatus, TenderResult, AwardedTenderedProject, ProcurementProjectLotID, , , )","(ContractFolderStatus, TenderingProcess, TenderSubmissionDeadlinePeriod, , , , )","(ContractFolderStatus, LocatedContractingParty, ParentLocatedParty, ParentLocatedParty, ParentLocatedParty, PartyName, Name)",deleted_on
file name,id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PlataformasAgregadasSinMenores_20210108_030012.atom,https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/6855054,342,Id licitación: 202100053; Órgano de Contratac...,Desarrollo del Programa BERBALAGUN,2021-01-07 11:40:32.933000+00:00,202100053,PUB,Alcalde,Ayuntamiento de Barakaldo,Desarrollo del Programa BERBALAGUN,2.0,29000.00,11455.00,80400000,ES213,NaN,999.0,2021-01-21,13:00:00,DOC_CN,Perfil del contratante,2021-01-07,2021-02-15,2021-12-30,Ayuntamiento de Barakaldo,PCAP BERBALAGUN 2021 signed.pdf,https://contratacion.euskadi.eus/w32-kpeperfi/...,BERBALAGUN - PLEGU TEKNIKOAK signed.pdf,https://contratacion.euskadi.eus/w32-kpeperfi/...,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,2021-01-21 13:00:00+00:00,<NA>,2021-01-07 13:40:42.170000+01:00
PlataformasAgregadasSinMenores_20210302_

### Completeness

Another columns of interest: *URL entity*

In [ ]:
url_entity_col = dlsproc.hier.pad_col_levels(only_last_update_df, ['ContractFolderStatus', 'LocatedContractingParty', 'BuyerProfileURIID', '', '', '', ''])
url_entity_col

('ContractFolderStatus',
 'LocatedContractingParty',
 'BuyerProfileURIID',
 '',
 '',
 '',
 '')

How many rows provide a non-null value for the entity URL?

In [ ]:
only_last_update_df[url_entity_col].notna().sum()

13755

What are the most repeated URLs?

In [ ]:
only_last_update_df.groupby(url_entity_col).size().sort_values()

(ContractFolderStatus, LocatedContractingParty, BuyerProfileURIID, , , , )
http://www.carm.es/neweb2/servlet/integra.serv...       1
https://contractaciopublica.gencat.cat/ecofin_...       1
https://contractaciopublica.gencat.cat/ecofin_...       1
https://contractaciopublica.gencat.cat/ecofin_...       1
https://contractaciopublica.gencat.cat/ecofin_...       1
                                                     ... 
https://www.larioja.org/contratacion-publica/e...     275
http://www.madrid.org/cs/Satellite?cid=1205761...     277
https://contractaciopublica.gencat.cat/ecofin_...     343
https://contractaciopublica.gencat.cat/ecofin_...     364
http://www.madrid.org/cs/Satellite?cid=1204201...    1629
Length: 1203, dtype: int64

A new column with the (internet) domain is added

In [ ]:
only_last_update_df['domain'] = dlsproc.parse.domain(only_last_update_df)

In [ ]:
only_last_update_df['domain']

file name                                            entry
PlataformasAgregadasSinMenores_20210101_030039.atom  294      contratacion.euskadi.eus
                                                     291                          <NA>
                                                     288                          <NA>
                                                     282                          <NA>
                                                     281                          <NA>
                                                                        ...           
PlataformasAgregadasSinMenores.atom                  4                            <NA>
                                                     3                            <NA>
                                                     2                            <NA>
                                                     1                            <NA>
                                                     0                 

It is used for grouping

In [ ]:
domain_grouped_only_last_update_df = only_last_update_df.groupby('domain', dropna=False)

The following groups (*domain*s) are present, each one with the indicated number of entries

In [ ]:
domain_grouped_only_last_update_df.size()

domain
contractaciopublica.gencat.cat    12092
contratacion.euskadi.eus           3812
place-agregacion.carm.es             32
www.asturias.es                    1739
www.bilbao.eus                      171
www.carm.es                           1
www.contratacion.euskadi.eus       4786
www.contratosdegalicia.gal         1160
www.larioja.org                     275
www.madrid.org                     4787
NaN                               35165
dtype: int64

For the sake of convenience

In [ ]:
domain_col = dlsproc.hier.pad_col_levels(outsiders_df, ['domain'])
domain_col

('domain', '', '', '', '', '', '')

For every group we ascertain which elements (across every column) are not null (still a `pd.DataFrame`), and then summarize it (one value per group and column) with `mean` (ratio of filled-in values)

In [ ]:
filled_in_ratio_df = domain_grouped_only_last_update_df.apply(
    lambda x: x.notnull().mean()).drop(domain_col, axis=1)
    # lambda x: x.notnull().mean()).drop(dlsproc.hier.pad_col_levels(only_last_update_df, ['domain']), axis=1)
filled_in_ratio_df

id summary title updated  \
                                                            
                                                            
                                                            
                                                            
                                                            
                                                            
domain                                                      
contractaciopublica.gencat.cat  1.0     1.0   1.0     1.0   
contratacion.euskadi.eus        1.0     1.0   1.0     1.0   
place-agregacion.carm.es        1.0     1.0   1.0     1.0   
www.asturias.es                 1.0     1.0   1.0     1.0   
www.bilbao.eus                  1.0     1.0   1.0     1.0   
www.carm.es                     1.0     1.0   1.0     1.0   
www.contratacion.euskadi.eus    1.0     1.0   1.0     1.0   
www.contratosdegalicia.gal      1.0     1.0   1.0     1.0   
www.larioja.org                 1.0     1.0   1.0     1.0   
www.madrid.org                  1.0     1.0   1.0     1.0   
NaN                             1.0     1.0   1.0     1.0   

                               ContractFolderStatus                           \
                                   ContractFolderID ContractFolderStatusCode   
                                                                               
                                                                               
                                                                               
                                                                               
                                                                               
domain                                                                         
contractaciopublica.gencat.cat             0.999835                      1.0   
contratacion.euskadi.eus                   1.000000                      1.0   
place-agregacion.carm.es                   1.000000                      1.0   
www.asturias.es                            1.000000                      1.0   
www.bilbao.eus                             1.000000                      1.0   
www.carm.es                                1.000000                      1.0   
www.contratacion.euskadi.eus               1.000000                      1.0   
www.contratosdegalicia.gal                 1.000000                      1.0   
www.larioja.org                            1.000000                      1.0   
www.madrid.org                             1.000000                      1.0   
NaN                                        0.995507                      1.0   

                                                                           \
                               LocatedContractingParty                      
                                                 Party ParentLocatedParty   
                                             PartyName          PartyName   
                                                  Name               Name   
                                                                            
                                                                            
domain                                                                      
contractaciopublica.gencat.cat                     1.0           0.999917   
contratacion.euskadi.eus                           1.0           1.000000   
place-agregacion.carm.es                           1.0           1.000000   
www.asturias.es                                    1.0           0.000000   
www.bilbao.eus                                     1.0           1.000000   
www.carm.es                                        1.0           1.000000   
www.contratacion.euskadi.eus                       1.0           1.000000   
www.contratosdegalicia.gal                         1.0           0.000000   
www.larioja.org                                    1.0           1.000000   
www.madrid.org                             

More readable (notice the ordering of the index)

In [ ]:
tidy_filled_in_ratio_df = filled_in_ratio_df.sort_index(axis=1).T
tidy_filled_in_ratio_df

domain                                                                                                                                                        contractaciopublica.gencat.cat  \
ContractFolderStatus ContractFolderID                                                                                                                                               0.999835   
                     ContractFolderStatusCode                                                                                                                                       1.000000   
                     LegalDocumentReference     Attachment                          ExternalReference                      URI                                                      0.434916   
                                                ID                                                                                                                                  0.434916   
                     LocatedContractingParty    BuyerProfileURIID                                                                                                                   0.861148   
                                                ParentLocatedParty                  ParentLocatedParty                     ParentLocatedParty PartyName Name                        0.000000   
                                                                                                                           PartyName          Name                                  0.000000   
                                                                                    PartyName                              Name                                                     0.999917   
                                                Party                               PartyIdentification                    ID                                                       0.000083   
                                                                                    PartyName                              Name                                                     1.000000   
                     ProcurementProject         BudgetAmount                        EstimatedOverallContractAmount                                                                  1.000000   
                                                                                    TaxExclusiveAmount                                                                              1.000000   
                                                Name                                                                                                                                1.000000   
                                                PlannedPeriod                       DurationMeasure                                                                                 0.904152   
                                                                                    EndDate                                                                                         0.095270   
                                                                                    StartDate                                                                                       0.095270   
                                                RealizedLocation                    CountrySubentityCode                                                                            0.999090   
                                                RequiredCommodityClassification     ItemClassificationCode                                                                          0.988505   
                                                TypeCode                                                                                                                            1.000000   
                     TechnicalDocumentReference Attachment                          ExternalReference                      URI                                  

It is saved to a spreadsheet

In [ ]:
tidy_filled_in_ratio_df.to_excel('outsiders_filled_in.xlsx')

#### GenCat

For the sake of convenience

In [ ]:
id_col = dlsproc.hier.pad_col_levels(outsiders_df, ['id'])
id_col

('id', '', '', '', '', '', '')

Entries for *Generalitat de Catalunya*

In [ ]:
gencat_df = only_last_update_df[only_last_update_df['domain'] == 'contractaciopublica.gencat.cat']
gencat_df.head(2)

id  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20210105_030039.... 352    https://contrataciondelestado.es/sindicacion/P...   
                                                   343    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                    summary  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20210105_030039.... 352    Id licitación: 001/20001709; Órgano de Contrat...   
                                                   343    Id licitación: X2020025214; Órgano de Contrata...   

                                                                                                      title  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20210105_030039.... 352    Serveis de Manteniment i subministrament de le...   
                                                   343    És objecte de la venda, pel procediment obert ...   

                                                                                  updated  \
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
file name                                          entry                                    
PlataformasAgregadasSinMenor

How many entries from *GenCat*

In [ ]:
len(gencat_df)

12092

*GenCat* filling of both *Entity* and *WinningParty*. Notice the **data being explored are referred to the last update** meaning that a ratio of 0 for some `file name` and column only entails that the entries *that were updated for the last time in that file* (and only those) all have the corresponding column empty.

In [ ]:
filename_notna_ratio = gencat_df[[url_entity_col, winner_col]].groupby('file name').apply(lambda x: x.notna().mean())
filename_notna_ratio

ContractFolderStatus  \
                                                   LocatedContractingParty   
                                                         BuyerProfileURIID   
                                                                             
                                                                             
                                                                             
                                                                             
file name                                                                    
PlataformasAgregadasSinMenores_20210105_030039....                     0.0   
PlataformasAgregadasSinMenores_20210106_030028....                     0.0   
PlataformasAgregadasSinMenores_20210108_030012....                     0.0   
PlataformasAgregadasSinMenores_20210108_030012_...                     0.0   
PlataformasAgregadasSinMenores_20210109_030014....                     0.0   
..                                                                     ...   
PlataformasAgregadasSinMenores_20211229_030012....                     1.0   
PlataformasAgregadasSinMenores_20211229_030012_...                     1.0   
PlataformasAgregadasSinMenores_20211230_030012....                     1.0   
PlataformasAgregadasSinMenores_20211231_030012....                     1.0   
PlataformasAgregadasSinMenores_20211231_030012_...                     1.0   

                                                                 
                                                   TenderResult  
                                                   WinningParty  
                                                      PartyName  
                                                           Name  
                                                                 
                                                                 
file name                                                        
PlataformasAgregadasSinMenores_20210105_030039....     0.000000  
PlataformasAgregadasSinMenores_20210106_030028....     0.000000  
PlataformasAgregadasSinMenores_20210108_030012....     0.000000  
PlataformasAgregadasSinMenores_20210108_030012_...     0.000000  
PlataformasAgregadasSinMenores_20210109_030014....     0.000000  
..                                                          ...  
PlataformasAgregadasSinMenores_20211229_030012....     0.600000  
PlataformasAgregadasSinMenores_20211229_030012_...     0.471698  
PlataformasAgregadasSinMenores_20211230_030012....     0.534884  
PlataformasAgregadasSinMenores_20211231_030012....     0.582645  
PlataformasAgregadasSinMenores_20211231_030012_...     0.583333  

[414 rows x 2 columns]

##### Entity

Files (given by the index) in which *GenCat* has not a single URL filled in, neither have the rest

In [ ]:
only_last_update_df.loc[filename_notna_ratio.index[filename_notna_ratio[url_entity_col] == 0]][url_entity_col].isna().all()

True

Files in which *GenCat* has *some* but not all URLs filled in

In [ ]:
filename_notna_ratio[(filename_notna_ratio[url_entity_col] > 0) & (filename_notna_ratio[url_entity_col] < 1.)]

ContractFolderStatus  \
                                                   LocatedContractingParty   
                                                         BuyerProfileURIID   
                                                                             
                                                                             
                                                                             
                                                                             
file name                                                                    
PlataformasAgregadasSinMenores_20211020_030012_...                     0.5   

                                                                 
                                                   TenderResult  
                                                   WinningParty  
                                                      PartyName  
                                                           Name  
                                                                 
                                                                 
file name                                                        
PlataformasAgregadasSinMenores_20211020_030012_...       0.4375

##### Winner

Maximum coverage

In [ ]:
filename_notna_ratio[winner_col].max()

0.8928571428571429

Unlike for the *Buyer* field above, files (given by the index) in which *GenCat* has not a single *Winner* filled in have entries filled in by other organizations

In [ ]:
only_last_update_df.loc[filename_notna_ratio.index[filename_notna_ratio[winner_col] == 0]][winner_col].isna().all()

False

Files in which *GenCat* has 0 coverage for *Winner*

In [ ]:
gencat_winner_zero_coverage = filename_notna_ratio[filename_notna_ratio[winner_col] == 0]
gencat_winner_zero_coverage

ContractFolderStatus  \
                                                   LocatedContractingParty   
                                                         BuyerProfileURIID   
                                                                             
                                                                             
                                                                             
                                                                             
file name                                                                    
PlataformasAgregadasSinMenores_20210105_030039....                     0.0   
PlataformasAgregadasSinMenores_20210106_030028....                     0.0   
PlataformasAgregadasSinMenores_20210108_030012....                     0.0   
PlataformasAgregadasSinMenores_20210108_030012_...                     0.0   
PlataformasAgregadasSinMenores_20210109_030014....                     0.0   
..                                                                     ...   
PlataformasAgregadasSinMenores_20211015_030012_...                     0.0   
PlataformasAgregadasSinMenores_20211016_030014....                     0.0   
PlataformasAgregadasSinMenores_20211016_030014_...                     0.0   
PlataformasAgregadasSinMenores_20211019_030015....                     0.0   
PlataformasAgregadasSinMenores_20211019_030015_...                     0.0   

                                                                 
                                                   TenderResult  
                                                   WinningParty  
                                                      PartyName  
                                                           Name  
                                                                 
                                                                 
file name                                                        
PlataformasAgregadasSinMenores_20210105_030039....          0.0  
PlataformasAgregadasSinMenores_20210106_030028....          0.0  
PlataformasAgregadasSinMenores_20210108_030012....          0.0  
PlataformasAgregadasSinMenores_20210108_030012_...          0.0  
PlataformasAgregadasSinMenores_20210109_030014....          0.0  
..                                                          ...  
PlataformasAgregadasSinMenores_20211015_030012_...          0.0  
PlataformasAgregadasSinMenores_20211016_030014....          0.0  
PlataformasAgregadasSinMenores_20211016_030014_...          0.0  
PlataformasAgregadasSinMenores_20211019_030015....          0.0  
PlataformasAgregadasSinMenores_20211019_030015_...          0.0  

[318 rows x 2 columns]

Indeed...

In [ ]:
assert gencat_df.loc[gencat_winner_zero_coverage.index, winner_col].isna().all()

What about overall (not only *GenCat*)?

In [ ]:
only_last_update_df.loc[gencat_winner_zero_coverage.index, winner_col].isna().all()

False

An example of file with zero coverage

In [ ]:
gencat_df.loc['PlataformasAgregadasSinMenores_20210105_030039.atom', id_col].tolist()

['https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/6578285',
 'https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/6842544',
 'https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/6745938',
 'https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/6770807']

A couple of example files

In [ ]:
gencat_df.loc[gencat_winner_zero_coverage.index[:2]]

id  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20210105_030039.... 352    https://contrataciondelestado.es/sindicacion/P...   
                                                   343    https://contrataciondelestado.es/sindicacion/P...   
                                                   342    https://contrataciondelestado.es/sindicacion/P...   
                                                   173    https://contrataciondelestado.es/sindicacion/P...   
PlataformasAgregadasSinMenores_20210106_030028.... 465    https://contrataciondelestado.es/sindicacion/P...   
                                                   60     https://contrataciondelestado.es/sindicacion/P...   

                                                                                                    summary  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20210105_030039.... 352    Id licitación: 001/20001709; Órgano de Contrat...   
                                                   343    Id licitación: X2020025214; Órgano de Contrata...   
                                                   342    Id licitación: 7/2020/COSSE; Órgano de Contrat...   
                                                   173    Id licitación: 2020-358/1403; Órgano de Contra...   
PlataformasAgregadasSinMenores_20210106_030028.... 465    Id licitación: 20004065; Órgano de Contratació...   
                                                   60     Id licitación: 1215/2020; Órgano de Contrataci...   

                                                                                                      title  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20210105_030039.... 352    Serveis de Manteniment i subministrament de le...

In [ ]:
# outsiders_df.loc[filename_notna_ratio[filename_notna_ratio[winner_col] == 0].index, winner_col].isna()

Files in which *GenCat* has *some* but not all *Winner*s filled in

In [ ]:
filename_notna_ratio[(filename_notna_ratio[winner_col] > 0) & (filename_notna_ratio[winner_col] < 1.)]

ContractFolderStatus  \
                                                   LocatedContractingParty   
                                                         BuyerProfileURIID   
                                                                             
                                                                             
                                                                             
                                                                             
file name                                                                    
PlataformasAgregadasSinMenores_20211020_030012....                     1.0   
PlataformasAgregadasSinMenores_20211020_030012_...                     0.5   
PlataformasAgregadasSinMenores_20211021_030015....                     1.0   
PlataformasAgregadasSinMenores_20211021_030015_...                     1.0   
PlataformasAgregadasSinMenores_20211022_030030....                     1.0   
..                                                                     ...   
PlataformasAgregadasSinMenores_20211229_030012....                     1.0   
PlataformasAgregadasSinMenores_20211229_030012_...                     1.0   
PlataformasAgregadasSinMenores_20211230_030012....                     1.0   
PlataformasAgregadasSinMenores_20211231_030012....                     1.0   
PlataformasAgregadasSinMenores_20211231_030012_...                     1.0   

                                                                 
                                                   TenderResult  
                                                   WinningParty  
                                                      PartyName  
                                                           Name  
                                                                 
                                                                 
file name                                                        
PlataformasAgregadasSinMenores_20211020_030012....     0.892857  
PlataformasAgregadasSinMenores_20211020_030012_...     0.437500  
PlataformasAgregadasSinMenores_20211021_030015....     0.736264  
PlataformasAgregadasSinMenores_20211021_030015_...     0.673469  
PlataformasAgregadasSinMenores_20211022_030030....     0.824176  
..                                                          ...  
PlataformasAgregadasSinMenores_20211229_030012....     0.600000  
PlataformasAgregadasSinMenores_20211229_030012_...     0.471698  
PlataformasAgregadasSinMenores_20211230_030012....     0.534884  
PlataformasAgregadasSinMenores_20211231_030012....     0.582645  
PlataformasAgregadasSinMenores_20211231_030012_...     0.583333  

[96 rows x 2 columns]

---

#### Xunta de Galicia

Entries for *Xunta de Galicia*

In [ ]:
gal_df = only_last_update_df[only_last_update_df['domain'] == 'www.contratosdegalicia.gal']
gal_df.head(2)

id  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20211020_030012_... 80     https://contrataciondelestado.es/sindicacion/P...   
PlataformasAgregadasSinMenores_20211020_030012.... 466    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                    summary  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20211020_030012_... 80     Id Licitación: PcPG/2021/802999, Órgano de Con...   
PlataformasAgregadasSinMenores_20211020_030012.... 466    Id Licitación: PcPG/2019/616022, Órgano de Con...   

                                                                                                      title  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20211020_030012_... 80                   Saneamiento en camino de la Carousa   
PlataformasAgregadasSinMenores_20211020_030012.... 466    Proyecto de ahorro y eficencia energética en e...   

                                                                                  updated  \
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
file name                                          entry                                    
PlataformasAgregadasSinMenor

*Xunta de Galicia* **always** provides the URL

In [ ]:
gal_df[url_entity_col].notna().all()

True

### Naming

A mapping between columns and *human-readable* fields can be found [here](***REMOVED***). The latter was processed in `naming.ipynb` to get the file below.

In [ ]:
input_file = pathlib.Path.cwd() / 'PLACE.yaml'
assert input_file.exists()

It provides (as a `dict`) a mapping from *human-readable* names to (maybe nested) fields in *Atom* files

In [ ]:
with open(input_file) as yaml_data:
    data_scheme = yaml.load(yaml_data, Loader=yaml.FullLoader)
{k: data_scheme[k] for k in list(data_scheme)[:5]}

{'id': ['id', nan, nan, nan, nan, nan, nan],
 'summary': ['summary', nan, nan, nan, nan, nan, nan],
 'title': ['title', nan, nan, nan, nan, nan, nan],
 'updated': ['updated', nan, nan, nan, nan, nan, nan],
 'Número de Expediente': ['ContractFolderStatus',
  'ContractFolderID',
  nan,
  nan,
  nan,
  nan,
  nan]}

The mapping can be exploited to access the columns of the `pd.DataFrame` using more natural names

In [ ]:
outsiders_df[dlsproc.hier.pad_col_levels(outsiders_df, data_scheme['Número de Expediente'], denan=True)]

file name                                              entry
PlataformasAgregadasSinMenores.atom                    0                       3064_23/2021
                                                       1        3051_CONT_SERVICIOS/2021/40
                                                       2                       3091_21/2021
                                                       3                        3069_1/2021
                                                       4                    8044_30146-0-21
                                                                           ...             
PlataformasAgregadasSinMenores_20211231_030012_1.atom  463                          2021.63
                                                       464                          2021.67
                                                       465                     C02/015/2021
                                                       466                    EJIE-146-2021
                   

The inverse of the above mapping **turning nan's into empty strings**

In [ ]:
inv_data_scheme = {tuple([e if pd.notna(e) else '' for e in v]): k for k, v in data_scheme.items()}
{k: inv_data_scheme[k] for k in list(inv_data_scheme)[:5]}

{('id', '', '', '', '', '', ''): 'id',
 ('summary', '', '', '', '', '', ''): 'summary',
 ('title', '', '', '', '', '', ''): 'title',
 ('updated', '', '', '', '', '', ''): 'updated',
 ('ContractFolderStatus',
  'ContractFolderID',
  '',
  '',
  '',
  '',
  ''): 'Número de Expediente'}

Every colum in the `pd.DataFrame` is looked up in the *inverse* mapping. If present, the column is renamed accordingly, otherwise the new name is simply the concatenation of all the (nested) labels

In [ ]:
new_names = []
unmapped_names = []
for c in outsiders_df.columns:
    # if the columns is found in the inverse mapping...
    if c in inv_data_scheme:
        # ...the given name is used
        new_names.append(inv_data_scheme[c])
    # if the columns is NOT found in the inverse mapping...
    else:
        # ...the new name is obtained by contatenating the individual components
        new_names.append(dlsproc.structure.nested_tags_separator.join([e for e in c if e != '']))
        
        # it is also recorded in its own list
        unmapped_names.append(new_names[-1])
new_names[:15]

['id',
 'summary',
 'title',
 'updated',
 'Número de Expediente',
 'Estado',
 'Nombre',
 'Ubicación orgánica',
 'Objeto del Contrato',
 'Tipo de Contrato',
 'Valor estimado del contrato',
 'Presupuesto base sin impuestos',
 'Clasificación CPV',
 'Código de Subentidad Nacional',
 'Plazo de Ejecución (Duración)']

In [ ]:
# ['ContractFolderStatus', 'TenderingProcess','TenderSubmissionDeadlinePeriod','EndDate']

Columns that were not mapped to a *human-readable* name

In [ ]:
print(*unmapped_names, sep='\n\n')

ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - ParentLocatedParty - PartyName - Name

ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - ParentLocatedParty - ParentLocatedParty - PartyName - Name

domain


Columns of the `pd.DataFrame` are renamed with *plain* names (`pd.MultiIndex` is gone)

In [ ]:
outsiders_df.columns = new_names

The column belows, e.g., mixes `list`s and elemental values

In [ ]:
outsiders_df['Clasificación CPV']

file name                                              entry
PlataformasAgregadasSinMenores.atom                    0                                        77310000
                                                       1                                        71241000
                                                       2                                        45231400
                                                       3        [45231300, 45213260, 45112000, 44611600]
                                                       4                                        45200000
                                                                                  ...                   
PlataformasAgregadasSinMenores_20211231_030012_1.atom  463                                      45210000
                                                       464                                      22200000
                                                       465                                      805

This must be handled with care (see [this post](http://www.legendu.net/misc/blog/python-complicated-data-types/))

Saving to *parquet*

In [ ]:
homogeneous_outsiders_df = dlsproc.io.homogenize_multivalued(outsiders_df)
homogeneous_outsiders_df

id  \
file name                                          entry                                                      
PlataformasAgregadasSinMenores.atom                0      https://contrataciondelestado.es/sindicacion/P...   
                                                   1      https://contrataciondelestado.es/sindicacion/P...   
                                                   2      https://contrataciondelestado.es/sindicacion/P...   
                                                   3      https://contrataciondelestado.es/sindicacion/P...   
                                                   4      https://contrataciondelestado.es/sindicacion/P...   
...                                                                                                     ...   
PlataformasAgregadasSinMenores_20211231_030012_... 463    https://contrataciondelestado.es/sindicacion/P...   
                                                   464    https://contrataciondelestado.es/sindicacion/P...   
                                                   465    https://contrataciondelestado.es/sindicacion/P...   
                                                   466    https://contrataciondelestado.es/sindicacion/P...   
                                                   467    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                    summary  \
file name                                          entry                                                      
PlataformasAgregadasSinMenores.atom                0      Id licitación: 3064_23/2021; Órgano de Contrat...   
                                                   1      Id licitación: 3051_CONT_SERVICIOS/2021/40; Ór...   
                                                   2      Id licitación: 3091_21/2021; Órgano de Contrat...   
                                                   3      Id licitación: 3069_1/2021; Órgano de Contrata...   
                                                   4      Id licitación: 8044_30146-0-21; Órgano de Cont...   
...                                                                                                     ...   
PlataformasAgregadasSinMenores_20211231_030012_... 463    Id licitación: 2021.63; Órgano de Contratació...   
                                                   464    Id licitación: 2021.67; Órgano de Contratació...   
                                                   465    Id licitación: C02/015/2021; Órgano de Contra...   
                                                   466    Id licitación: EJIE-146-2021; Órgano de Contr...   
                                                   467    Id licitación: 2021101CO110; Órgano de Contra...   

                                                                                                      title  \
file name                                          entry                                                      
PlataformasAgregadasSinMenores.atom                0      Contrato del Servicio para el desarrollo de la...   
                                                   1      REDACCIÓN DEL PLAN ESTRATÉGICO DE ACCESIBILIDA...   
                                                   2      Contratación de obra de línea subterránea en A...   
                                                   3      Construcción de depósito regulador de 100m3 en...   
                                                   4          Renovación de la cubierta de edificio docente   
...                                                                                                     ...   
PlataformasAgregadasSinMenores_20211231_030012_... 463    Contratación de la obra de reforma interior se...   
                                                   464    Contratación del suministro de prensa escrita ...   
                                                   465    Organización de cursos de inglés, francés y al...

In [ ]:
homogeneous_outsiders_df['Medio de Publicación'].apply(dlsproc.io.cast_list_to_floats_or_strs)

file name                                              entry
PlataformasAgregadasSinMenores.atom                    0                                                  [DOUE]
                                                       1                                [Perfil del contratante]
                                                       2                                [Perfil del contratante]
                                                       3                          [DOUE, Perfil del contratante]
                                                       4                                [Perfil del contratante]
                                                                                      ...                       
PlataformasAgregadasSinMenores_20211231_030012_1.atom  463                              [Perfil del contratante]
                                                       464                              [Perfil del contratante]
                                   

In [ ]:
homogeneous_outsiders_df['Importe total ofertado (sin impuestos)'].apply(dlsproc.io.cast_list_to_floats_or_strs)

file name                                              entry
PlataformasAgregadasSinMenores.atom                    0              [nan]
                                                       1              [nan]
                                                       2              [nan]
                                                       3              [nan]
                                                       4              [nan]
                                                                   ...     
PlataformasAgregadasSinMenores_20211231_030012_1.atom  463            [nan]
                                                       464            [nan]
                                                       465            [nan]
                                                       466      [716038.99]
                                                       467            [nan]
Name: Importe total ofertado (sin impuestos), Length: 208204, dtype: object

A list with the names of columns that are *multivalued*

In [ ]:
multivalued_columns = dlsproc.structure.multivalued_columns(homogeneous_outsiders_df)
multivalued_columns

['Clasificación CPV',
 'Tipo de Anuncio',
 'Medio de Publicación',
 'Fecha de Publicación',
 'Resultado',
 'Número de Licitadores Participantes',
 'Identificador (+ Tipo: mod schemeName)',
 'Nombre del Adjudicatario',
 'Importe total ofertado (sin impuestos)',
 'Lote']

The same type is enforced in all the `list`s (across rows and columns) in *multivalued* columns

In [ ]:
homogeneous_outsiders_df[multivalued_columns] = homogeneous_outsiders_df[multivalued_columns].applymap(dlsproc.io.cast_list_to_floats_or_strs)

Afterwards, the same type for the elements in the list is enforced across every **individual** *multivalued* column

In [ ]:
homogeneous_outsiders_df[multivalued_columns] = homogeneous_outsiders_df[multivalued_columns].apply(dlsproc.io.cast_multivalued_series_to_common_type, axis='index')

Some checks:

In [ ]:
homogeneous_outsiders_df['Identificador (+ Tipo: mod schemeName)'].apply(lambda x: type(x[0]))

file name                                              entry
PlataformasAgregadasSinMenores.atom                    0        <class 'str'>
                                                       1        <class 'str'>
                                                       2        <class 'str'>
                                                       3        <class 'str'>
                                                       4        <class 'str'>
                                                                    ...      
PlataformasAgregadasSinMenores_20211231_030012_1.atom  463      <class 'str'>
                                                       464      <class 'str'>
                                                       465      <class 'str'>
                                                       466      <class 'str'>
                                                       467      <class 'str'>
Name: Identificador (+ Tipo: mod schemeName), Length: 208204, dtype: object

### Saving

Metadata (right now, it seems it's not saved when writing the `pd.DataFrame` to disk)

In [ ]:
# stateful_outsiders_df.attrs['# duplicate deletes'] = n_duplicate_deletes

Saving to *parquet* (and maybe in the [Google Drive shared unit](https://drive.google.com/drive/folders/0ADw_MTEsA4fSUk9PVA))

In [ ]:
output_file = outsiders_output_file.name.with_suffix('.parquet')
# homogeneous_outsiders_df.to_parquet(output_file)
# only_last_update_df.to_parquet(output_file.with_stem(output_file.stem + '_most_recent'))
stateful_outsiders_df.to_parquet(output_file)

In [ ]:
restored_df = pd.read_parquet(output_file)
restored_df.head()

,summary,title,updated,Número de Expediente,Estado,Nombre,Ubicación orgánica,Objeto del Contrato,Tipo de Contrato,Valor estimado del contrato,Presupuesto base sin impuestos,Clasificación CPV,Código de Subentidad Nacional,Plazo de Ejecución (Duración),Tipo de Procedimiento,Presentación de Oferta (Fecha),Presentación de Oferta (Hora),Tipo de Anuncio,Medio de Publicación,Fecha de Publicación,Plazo de Ejecución (Comienzo),Plazo de Ejecución (Fin),ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - ParentLocatedParty - PartyName - Name,Pliego de cláusulas administrativas,Pliego de cláusulas administrativas (URI),Pliego de Prescripciones técnicas,Pliego de Prescripciones técnicas (URI),Resultado,Número de Licitadores Participantes,Identificador (+ Tipo: mod schemeName),Nombre del Adjudicatario,Importe total ofertado (sin impuestos),Presentación de Solicitudes (Fecha),Presentación de Solicitudes (Hora),URL perfil de contratante,ID,Lote,Presentación de Oferta,ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - ParentLocatedParty - ParentLocatedParty - PartyName - Name,domain,file name,entry,deleted_on
0,Id licitación: BE-2015-2059; Importe: 59999.00...,Servei de dinamització de l¿equipament cívic F...,2021-04-22 14:11:48.528000+00:00,BE-2015-2059,RES,Departament de Benestar Social I Família,<NA>,Servei de dinamització de l¿equipament cívic F...,2.0,59999.00,59999.00,[98133000.0],ES511,1.0,3.0,2015-06-26,02:00:00,"[DOC_CN, DOC_CAN_ADJ, DOC_FORM]","[Perfil del contratante, Perfil del contratant...","[2015-06-15, 2015-10-13, 2015-12-23]",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[9.0],[1.0],[G58422791],[ASSOCIACIÓ DE VEÏNS DE FONT DELS CAPELLANS],[59999.0],<NA>,<NA>,<NA>,A09002972,[nan],2015-06-26 02:00:00+00:00,<NA>,None,PlataformasAgregadasSinMenores_20210120_030025...,https://contrataciondelestado.es/sindicacion/P...,2021-01-19 00:11:01.869000+01:00
1,Id licitación: BE-2015-2854; Importe: 309112.9...,Servei de monitoratge de les activitats que es...,2021-04-22 08:11:26.837000+00:00,BE-2015-2854,RES,Departament de Benestar Social I Família,<NA>,Servei de monitoratge de les activitats que es...,2.0,803693.72,309112.97,[92000000.0],ES51,1.0,1.0,2015-07-30,02:00:00,"[DOC_CN, DOC_CAN_ADJ, DOC_FORM]","[Perfil del contratante, Perfil del contratant...","[2015-07-14, 2015-11-20, 2015-12-18]",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[9.0],[5.0],[R5800395E],[FUNDACIÓ PIA INSTITUT PERE TARRÉS D'EDUCACIÓ ...,[0.0],<NA>,<NA>,<NA>,A09002972,[nan],2015-07-30 02:00:00+00:00,<NA>,None,PlataformasAgregadasSinMenores_20210219_030030...,https://contrataciondelestado.es/sindicacion/P...,2021-02-18 00:11:08.608000+01:00
2,Id licitación: BE-2015-2852; Importe: 414141.8...,Servei de monitoratge de les activitats que es...,2021-04-22 08:11:27.336000+00:00,BE-2015-2852,RES,Departament de Benestar Social I Família,<NA>,Servei de monitoratge de les activitats que es...,2.0,1076768.82,414141.85,[92000000.0],ES511,1.0,1.0,2015-07-30,02:00:00,"[DOC_CN, DOC_CAN_ADJ, DOC_FORM]","[Perfil del contratante, Perfil del contratant...","[2015-07-14, 2015-11-20, 2015-12-18]",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[9.0],[4.0],[R5800395E],[FUNDACIÓ PIA INSTITUT PERE TARRÉS D'EDUCACIÓ ...,[0.0],<NA>,<NA>,<NA>,A09002972,[nan],2015-07-30 02:00:00+00:00,<NA>,None,PlataformasAgregadasSinMenores_20210219_030030...,https://contrataciondelestado.es/sindicacion/P...,2021-02-18 00:11:08.510000+01:00
3,Id licitación: BE-2015-2855; Importe: 237930.9...,Servei de monitoratge de les activitats que es...,2021-04-22 08:11:26.206000+00:00,BE-2015-2855,RES,Departament de Benestar Social I Família,<NA>,Servei de monitoratge de les activitats que es...,2.0,618620.50,237930.96,[92000000.0],ES51,1.0,1.0,2015-07-28,02:00:00,"[DOC_CN, DOC_CAN_ADJ, DOC_FORM]","[Perfil del contratante, Perfil del contratant...","[2015-07-20, 2015-11-19, 2015-12-23]",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[9.0],[4.0],[R5800395E],[FUNDACIÓ PIA INSTITUT PERE TARRÉS D'EDUCACIÓ ...,[0.0],<NA>,<NA>,<NA>,A09002

In [ ]:
len(restored_df)

64020

Saving to *feather*

In [ ]:
# output_file = outsiders_output_file.name.with_suffix('.feather')
# homogeneous_outsiders_df.reset_index().to_feather(output_file)